In [1]:
from collections import deque, defaultdict
from functools import lru_cache
from aoc_helpers import advent

advent.setup(2022, 7)
fin = advent.get_input()

[advent] Getting input for year 2022 day 7... done (from disk).


In [2]:
def parse_filesystem(fin):
    lines = deque(fin)
    fs    = defaultdict(list)
    path  = ()
    
    while lines:
        line = lines.popleft().split() #['$', 'cd', 'foo']
        command = line[1] #'cd' or 'ls'
        args    = line[2:] #dir name, file, or nothing
        
        if command == 'ls':
            while lines and not lines[0].startswith('$'):
                size = lines.popleft().split()[0]
                if size == 'dir': #ignore if it's a directory
                    continue
                fs[path].append(int(size)) #adds file size to contents of path
        else: #cd into different dir
            if args[0] == '..':
                path = path[:-1]
            else:
                new_path = path + (args[0],)
                fs[path].append(new_path)
                path = new_path
    return fs


@lru_cache(maxsize=None)
def directory_size(path):
    size = 0

    for subdir_or_size in fs[path]:
        if isinstance(subdir_or_size, int):
            size += subdir_or_size
        else:
            size += directory_size(subdir_or_size) #recursively calculates size

    return size
        
fs = parse_filesystem(fin)

In [3]:
dir_size = 0
used = directory_size(('/',))
free = 70000000 - used
need = 30000000 - free
min_size_to_free = used

for path in fs:
    size = directory_size(path)
    if size <= 100000:
        dir_size += size
    if size >= need and size < min_size_to_free:
        min_size_to_free = size
        
advent.print_answer(1, dir_size)
advent.print_answer(2, min_size_to_free)

Part 1: 1428881
Part 2: 10475598
